# <center>Deployment</center>

### Connect to the workspace

In [32]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.44.0 to work with avanade-airline-delays


Let's check if the model is registered.

In [33]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

airline_delays_classification_2 version: 1
	 Training context : Auto ML
	 Task : Classification
	 Objective : Avanade Challenge
	 AUC : 0.6865246130797511
	 Accuracy : 0.8259597752362156
	 F1_score : 0.7685567757525456


airline_delays_classification version: 1
	 Training context : Auto ML
	 Task : Classification
	 Objective : Avanade Challenge
	 AUC : 0.6865246130797511
	 Accuracy : 0.8259597752362156
	 F1_score : 0.7685567757525456




Model needs to be assigned to the variable in order to be deployed.

In [36]:
model = ws.models['airline_delays_classification']
model

Model(workspace=Workspace.create(name='avanade-airline-delays', subscription_id='a0f61829-e513-45af-b347-0542a63713cd', resource_group='moja_resource_grupa'), name=airline_delays_classification, id=airline_delays_classification:1, version=1, tags={'Training context': 'Auto ML', 'Task': 'Classification', 'Objective': 'Avanade Challenge'}, properties={'AUC': '0.6865246130797511', 'Accuracy': '0.8259597752362156', 'F1_score': '0.7685567757525456'})

Now, let's create a folder for the configuration files.

In [40]:
import os

# Create a folder for the deployment and model files
deployment_folder = './airlines_delays_deploy_dir'
model_folder = './airlines_delays_deploy_dir/Model'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_airlines.py'
script_path = os.path.join(deployment_folder,script_file)

./airlines_delays_deploy_dir folder created.


And the script.

In [44]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join('./airlines_delays_deploy_dir/Model', 'model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['on-time', 'delayed']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./airlines_delays_deploy_dir/score_airlines.py


### Create enviroment file

In [45]:
%%writefile $deployment_folder/deploy_env.yml
name: deploy_environment
dependencies:
- python=3.8.5
- pip:
  - azureml-train-automl-runtime==1.46.1
  - inference-schema
  - azureml-interpret==1.46.0
  - azureml-defaults==1.46.0
- numpy==1.21.6
- pandas==1.1.5
- scikit-learn==0.22.1
- py-xgboost==1.3.3
- fbprophet==0.7.1
- holidays==0.10.3
- psutil==5.9.0
channels:
- anaconda
- conda-forge

Overwriting ./airlines_delays_deploy_dir/deploy_env.yml


In [46]:
from azureml.core import Environment

deploy_env = Environment.from_conda_specification("deployt_env", deployment_folder + "/deploy_env.yml")

### Configuring the scoring environment

In [47]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=deploy_env)

### Configuring the web service container

In [48]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "airline-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-10-26 06:47:14+00:00 Creating Container Registry if not exists.
2022-10-26 06:47:16+00:00 Use the existing image.
2022-10-26 06:47:16+00:00 Generating deployment configuration.
2022-10-26 06:47:19+00:00 Submitting deployment to compute..
2022-10-26 06:47:23+00:00 Checking the status of deployment airline-service..
2022-10-26 06:57:18+00:00 Checking the status of inference endpoint airline-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Test the deployed service

This is the schema in which data has to be passed to the created service in order to receive prediction whether the flight can be deleyed or on-time. 

|DISTANCE|PRCP|SNOW|SNWD|TMAX|AWND|LATITUDE|LONGITUDE|NUMBER_OF_SEATS|DEP_TIME_AFTERNOON|DEP_TIME_EVENING|DEP_TIME_MORNING|DEP_TIME_NIGHT|DAY_OF_WEEK_1|DAY_OF_WEEK_2|DAY_OF_WEEK_3|DAY_OF_WEEK_4|DAY_OF_WEEK_5|DAY_OF_WEEK_6|DAY_OF_WEEK_7|
|-----|------|---|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|----|

Here, I want to see if data can reach the service and make a single prediction.

In [57]:
import json

x_new = [[0.016, 0.038, 0, 0, 0.417, 0.245, 0.472, -0.478, 0.148, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ]]
print ('Flight data: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Flight data: [0.016, 0.038, 0, 0, 0.417, 0.245, 0.472, -0.478, 0.148, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
on-time


Now let's get the endpoint's URI and check if it's possible to make HTTP requests.

In [58]:
endpoint = service.scoring_uri
print(endpoint)

http://846677fa-0f08-49fe-a96c-514e42465291.westeurope.azurecontainer.io/score


In [62]:
endpoint = 'http://846677fa-0f08-49fe-a96c-514e42465291.westeurope.azurecontainer.io/score'

In [64]:
import requests

x_new = [[0.016, 0.038, 0, 0, 0.417, 0.245, 0.472, -0.478, 0.148, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1 ],
         [0.032, 0.032, 1, 0, 0.317, 0.645, 0.377, -0.178, 0.162, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0 ]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Flight {}".format(x_new[i]), predicted_classes[i] )

Flight [0.016, 0.038, 0, 0, 0.417, 0.245, 0.472, -0.478, 0.148, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] on-time
Flight [0.032, 0.032, 1, 0, 0.317, 0.645, 0.377, -0.178, 0.162, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0] delayed


Looks like service has been successfully deployed and requests can be sent without any problem.